In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import norm
from scipy import stats

from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
# pfuheprf lfyys[]
train = pd.read_csv('train.csv', index_col=0, encoding='utf-8')

In [3]:
train.head()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
Id,,,,,,,,,,,,,,,,,,,
14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 14038 to 6306
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   DistrictId     10000 non-null  int64  
 1   Rooms          10000 non-null  float64
 2   Square         10000 non-null  float64
 3   LifeSquare     7887 non-null   float64
 4   KitchenSquare  10000 non-null  float64
 5   Floor          10000 non-null  int64  
 6   HouseFloor     10000 non-null  float64
 7   HouseYear      10000 non-null  int64  
 8   Ecology_1      10000 non-null  float64
 9   Ecology_2      10000 non-null  object 
 10  Ecology_3      10000 non-null  object 
 11  Social_1       10000 non-null  int64  
 12  Social_2       10000 non-null  int64  
 13  Social_3       10000 non-null  int64  
 14  Healthcare_1   5202 non-null   float64
 15  Helthcare_2    10000 non-null  int64  
 16  Shops_1        10000 non-null  int64  
 17  Shops_2        10000 non-null  object 
 18  Price   

In [5]:
# чистка выбросов, и предобратка датасета
train = train[train['Rooms'] < 10]
train = train[train['Square'] < 250]
train = train[train['HouseFloor'] < 100]

In [6]:
def preproc(df_input, dummy_features=None):

    df_output = df_input.copy()

    Square = dict(df_output.groupby('Rooms')['Square'].median())
    idx = (df_output['Square'] < 15) & (df_output['Rooms'] < 2)
    df_output.loc[idx, 'Square'] = df_output.loc[idx, 'Rooms'].apply(lambda x: Square[x])

    idx = (df_output['Square'] < 30) & (df_output['Rooms'] > 1)
    df_output.loc[idx, 'Square'] = df_output.loc[idx, 'Rooms'].apply(lambda x: Square[x])

    idx = (df_output['Square'] < df_output['LifeSquare'])
    df_output.loc[idx, 'Square'] = df_output.loc[idx, 'LifeSquare']

    LifeSquare = dict(df_output.groupby('Rooms')['LifeSquare'].median())
    idx = (df_output['LifeSquare'].isnull())
    df_output.loc[idx, 'LifeSquare'] = df_output.loc[idx, 'Rooms'].apply(lambda x: LifeSquare[x])

    idx = (df_output['Rooms'] < 1)
    df_output.loc[idx, 'KitchenSquare'] = 0.0

    KitchenSquare = dict(df_output.groupby('Rooms')['KitchenSquare'].median())
    idx = (df_output['Rooms'] > 0)
    df_output.loc[idx, 'KitchenSquare'] = df_output.loc[idx, 'Rooms'].apply(lambda x: KitchenSquare[x])

    idx = (df_output['HouseFloor'] < df_output['Floor'])
    df_output.loc[idx, 'HouseFloor'] = df_output.loc[idx, 'Floor']

    idx = (df_output['HouseYear'] >= 2019)
    df_output.loc[idx, 'HouseYear'] = 2019

    df_output['Healthcare_1'].fillna(df_output['Healthcare_1'].median(), inplace=True)

    df_output['Ecology_2'] = df_output['Ecology_2'].apply(lambda x: 1 if x == 'A' else 0)
    df_output['Ecology_3'] = df_output['Ecology_3'].apply(lambda x: 1 if x == 'A' else 0)
    df_output['Shops_2'] = df_output['Shops_2'].apply(lambda x: 1 if x == 'A' else 0)

    if dummy_features != None:
        df_output = pd.get_dummies(data=df_output, columns=dummy_features, drop_first=True)

    return df_output

In [7]:
train = preproc(train, dummy_features=None)

In [18]:
y = train['Price'].values
X = train.drop(labels=['DistrictId', 'Price'], axis=1)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

In [20]:
# нормализация
scaler = MinMaxScaler()

X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

In [21]:
rfr = RandomForestRegressor()
rfr.fit(X_train_sc, y_train)

RandomForestRegressor(max_depth=26, max_features=0.5, min_samples_split=5,
                      n_estimators=400, n_jobs=-1, random_state=42)

In [23]:
y_pred_train = rfr.predict(X_train_sc)
y_pred_test = rfr.predict(X_test_sc)

score_train = r2_score(y_train, y_pred_train)
score_test = r2_score(y_valid, y_pred_test)

print(f'train: {np.round(score_train, 4)}\ntest: {np.round(score_test, 4)}')

train: 0.9376
test: 0.7169


In [24]:
# после подбора модели и оптимальных гиперпараметров, обучаем модель на всех данных train
scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X)

model = RandomForestRegressor(
    n_estimators=400,
    max_depth=26,
    max_features=0.5,
    min_samples_leaf=1,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)


model.fit(X_scaled, y)

scores = cross_val_score(model, X_scaled, y, scoring='r2', cv=10, n_jobs=-1)

print(f'cv mean: {np.round(np.mean(scores), 4)}')
print(f'cv std:  {np.round(np.std(scores), 4)}')

cv mean: 0.7357
cv std:  0.0243


In [25]:
test_data = pd.read_csv('test.csv', index_col=0, encoding='utf-8')

In [26]:
test_data.head()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
Id,,,,,,,,,,,,,,,,,,
725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,B,B,11,2748,1,NaN,0,0,B
15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B
5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,4702.0,5,5,B
15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,B,B,23,4583,3,NaN,3,3,B
14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,B,B,2,629,1,NaN,0,0,A


In [27]:
test_data_pr = preproc(test_data, dummy_features=None)

In [28]:
X_test_data = test_data_pr.drop(labels=['DistrictId'], axis=1)

In [29]:
X_test_data.head()

,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
Id,,,,,,,,,,,,,,,,,
725,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,0,0,11,2748,1,900.0,0,0,0
15856,2.0,69.263183,32.830730,6.0,6,6.0,1977,0.075779,0,0,6,1437,3,900.0,0,2,0
5480,1.0,40.117689,15.948246,6.0,2,5.0,1909,0.000000,0,0,30,7538,87,4702.0,5,5,0
15664,2.0,73.046609,51.940842,6.0,22,22.0,2007,0.101872,0,0,23,4583,3,900.0,3,3,0
14275,1.0,47.527111,43.387569,6.0,17,17.0,2017,0.072158,0,0,2,629,1,900.0,0,0,1


In [30]:
X_test_data_sc = scaler.transform(X_test_data)

In [31]:

y_pred_test = model.predict(X_test_data_sc)

In [33]:
predictions = pd.DataFrame({
    'Id': test_data.index,
    'Price': y_pred_test
})

In [ ]:
predictions.head()

,Id,Price
0,725,159716.003524
1,15856,205133.085284
2,5480,272968.271945
3,15664,350647.142002
4,14275,136769.439246


In [34]:
predictions.to_csv('DVatolin_predictions.csv', sep=',', index=False, encoding='utf-8')
